In [6]:
# test_xi_init_real.py
import numpy as np
from collections import Counter

from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models

# ---- Adjust these ----
BASE_DIR = "../data/Fantasy-Premier-League"   # path to the vaastav repo clone root
SEASONS  = ["2023-24"]                        # start with one season you have features for
START_GW = 1                                  # GW to start the episode
models = load_models("../models/rand_forest/classifiers")


env = FPLEnv(
    load_season_fn=load_season_fn,  # now returns season_ctx incl. predictor (if you added it)
    load_gw_fn=load_gw_fn,          # forwards predictor to the pool builder
    seasons=["2023-24"],
    base_dir="../data/Fantasy-Premier-League",
    start_gw=2,                     # if you want transfers to start at GW2
    budget=100.0,
    temperature=0.8,
    transfer_hit=-4.0,
    max_free_transfers=5,
    models=models,                  # <-- pass the 4 models here
)
obs, info = env.reset()


In [1]:
# test_env_masks_and_flow.py
import numpy as np

from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models

BASE_DIR = "../data/Fantasy-Premier-League"
SEASONS  = ["2023-24"]
MODELS   = load_models("../models/rand_forest/classifiers")

def first_true_idx(arr, avoid=set()):
    for i, v in enumerate(arr):
        if v and i not in avoid:
            return i
    return None

def run():
    env = FPLEnv(
        load_season_fn=load_season_fn,
        load_gw_fn=load_gw_fn,
        seasons=SEASONS,
        base_dir=BASE_DIR,
        start_gw=2,
        budget=100.0,
        temperature=0.8,
        transfer_hit=-4.0,
        max_free_transfers=5,
        models=MODELS,
    )

    obs, info = env.reset()
    print("Reset ok. Phase:", info["action"], info.get("phase"))
    mask_out = info["action_mask_out"]
    mask_in  = info["action_mask_in"]

    # --- Phase 1 masks ---
    assert info["phase"] == 1
    assert mask_out.shape == (16,), "mask_out shape"
    assert mask_in.shape  == (env.max_pool_size + 1,), "mask_in shape"
    assert mask_out[15] == 1, "skip-out must be allowed"
    assert mask_in[env.SKIP_IN] == 1, "skip-in must be allowed"
    assert mask_in[:env.pool_size].sum() == 0, "phase-1 mask_in should be skip-only"

    # 1) Phase-1 skip -> Phase-2 must be skip-only
    obs, reward, done, trunc, info = env.step(np.array([15, env.SKIP_IN], dtype=np.int64))
    assert not done
    assert info["phase"] == 2
    m2_out, m2_in = info["action_mask_out"], info["action_mask_in"]
    assert m2_out[15] == 1 and m2_out.sum() == 1, "after skip in phase-1, only skip-out is allowed in phase-2"
    assert m2_in[env.SKIP_IN] == 1 and m2_in.sum() == 1, "after skip in phase-1, only skip-in is allowed in phase-2"

    # 2) New GW: do a real transfer in phase-1 and check phase-2 constraints
    obs, info = env.reset()
    assert info["phase"] == 1
    m1_out, m1_in = info["action_mask_out"], info["action_mask_in"]

    # pick a legal out slot (not skip)
    out1 = first_true_idx(m1_out, avoid={15})
    assert out1 is not None, "need at least one legal out slot in phase-1"

    # compute phase-1 in choices: we need to ask env for phase-2 masks (it returns them after step)
    # take a placeholder in (skip) now; sanitizer will enforce realism later
    obs, reward, done, trunc, info = env.step(np.array([out1, env.SKIP_IN], dtype=np.int64))
    assert info["phase"] == 2

    # find a legal incoming for *phase-2* based on the chosen out1
    # IMPORTANT: we will pick out2 different from out1 and then sanitize 'inn' per chosen out2
    m2_out, _ = info["action_mask_out"], info["action_mask_in"]
    assert m2_out[out1] == 0, "cannot reuse the same out slot in phase-2"

    out2 = first_true_idx(m2_out, avoid={15, out1})
    if out2 is None:
        # If no second out is legal, second step must be skip and the env should accept it
        obs, reward, done, trunc, info = env.step(np.array([15, env.SKIP_IN], dtype=np.int64))
        print("No second transfer available; phase-2 skip accepted.")
    else:
        # Try to cheat and set an 'inn' equal to the first incoming (if any) or buy back sold player.
        # The sanitizer should force it to SKIP if illegal for out2.
        # We don't know the sold1/incoming1 from outside; just pick some index < pool_size and rely on sanitizer.
        inn2_try = 0 if env.pool_size > 0 else env.SKIP_IN

        obs, reward, done, trunc, info = env.step(np.array([out2, inn2_try], dtype=np.int64))
        assert info["phase"] == 1, "after phase-2, env should reset to phase-1 for next GW"
        print("Second transfer processed. Reward:", reward)

    # 3) Sanity: mask_in should mark all indices in (pool_size .. max_pool_size-1) as 0, except SKIP_IN.
    obs, info = env.reset()
    m1_out, m1_in = info["action_mask_out"], info["action_mask_in"]
    tail_sum = m1_in[env.pool_size:env.max_pool_size].sum()
    assert tail_sum == 0, "phantom indices must be 0"
    assert m1_in[env.SKIP_IN] == 1, "SKIP_IN must be 1"

    print("All mask & flow checks passed ✅")

if __name__ == "__main__":
    run()


Reset ok. Phase: init_gw2 1
No second transfer available; phase-2 skip accepted.
All mask & flow checks passed ✅


In [8]:
outcome = two_transfer_test(env)
print("Info:")
print(outcome[4])  # info contains the step details
print("next pool size", env.pool_size)

Proposed transfers:
  T1: OUT Luke Shaw  ->  IN Trent Alexander-Arnold
  T2: OUT Marcus Rashford  ->  IN Luis Díaz
Step info: {'season': '2023-24', 'current_gw': 3, 'team_xP_expected': 73.0, 'captain': 'William Saliba', 'vice_captain': 'Luis Díaz', 'xi_indices': [0, 6, 4, 2, 3, 5, 8, 7, 10, 9, 12], 'bank': 1.4000000000000004, 'free_transfers': 0, 'action': 'step', 'points_hit': -4.0}
Reward: 1.0
Info:
{'season': '2023-24', 'current_gw': 3, 'team_xP_expected': 73.0, 'captain': 'William Saliba', 'vice_captain': 'Luis Díaz', 'xi_indices': [0, 6, 4, 2, 3, 5, 8, 7, 10, 9, 12], 'bank': 1.4000000000000004, 'free_transfers': 0, 'action': 'step', 'points_hit': -4.0}
next pool size 80


In [9]:
print(env.current_gw)
env.squad.players

3


[Player(pid=597, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.0, features=(5.298056615700518, 4.376920756080059, 2.0, 2.75, 0.0, 0.4715776935396898, 100.0), pooling_metric=117188.0, xP=0.633804371865569),
 Player(pid=524, name='Alphonse Areola', pos='GK', team='West Ham', team_id=19, price=4.0, features=(6.701943384299483, 2.999043393329875, 3.0, 3.5, 0.0, 0.8259030868184272, 100.0), pooling_metric=186702.0, xP=0.6624064236457202),
 Player(pid=290, name='Trent Alexander-Arnold', pos='DEF', team='Liverpool', team_id=11, price=8.0, features=(1.0, 2.3, 2.0, 2.75, 0.0, 0.5196898566648355, 100.0), pooling_metric=83246.0, xP=0.6924762964528199),
 Player(pid=131, name='Pervis Estupiñán', pos='DEF', team='Brighton', team_id=5, price=5.2, features=(9.115396219599704, 13.249984914520386, 2.0, 3.25, 0.0, 0.9676660237773108, 100.0), pooling_metric=806778.0, xP=0.6670465370840241),
 Player(pid=368, name='John Stones', pos='DEF', team='Man City', team_id=13, price=5.5, features=

In [1]:
# test_random_start_window.py
import numpy as np
from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models

BASE_DIR = "../data/Fantasy-Premier-League"
SEASONS  = ["2023-24"]
MODELS   = load_models("../models/rand_forest/classifiers")

def run_once():
    env = FPLEnv(
        load_season_fn=load_season_fn,
        load_gw_fn=load_gw_fn,
        seasons=SEASONS,
        base_dir=BASE_DIR,
        start_gw=2,                # ignored when randomize_start=True
        budget=100.0,
        temperature=0.8,
        transfer_hit=-4.0,
        max_free_transfers=5,
        models=MODELS,
        randomize_start=True,
        max_episode_gws=12,
    )

    obs, info = env.reset()
    start_gw = info["start_gw"]
    end_gw   = info["end_gw"]
    assert 1 <= start_gw <= 33, f"start_gw out of range: {start_gw}"
    assert end_gw >= start_gw, "end_gw must be >= start_gw"
    assert end_gw - start_gw + 1 <= 12, "window longer than 12"
    assert end_gw <= env.total_gws, "end_gw beyond season length"

    steps = 0
    done = False
    while not done and steps < 20:  # 12 max by design, 20 is just a hard cap
        # choose the safe “skip, skip” from masks
        m_out, m_in = env._build_masks()
        # find the (skip, skip) choice for the raw env (out=15, in=SKIP_IN)
        action = np.array([15, env.SKIP_IN], dtype=np.int64)
        obs, rew, done, trunc, info = env.step(action)
        steps += (env.phase == 1)  # reward only applied on phase 2; count full GW steps after phase 2
        if done:
            assert env.current_gw > end_gw, "should terminate right after passing end_gw"
            break

    env.close()
    return start_gw, end_gw, steps

if __name__ == "__main__":
    for i in range(5):
        s, e, k = run_once()
        print(f"Run {i}: start={s}, end={e}, gw_steps≈{(e - s + 1)} OK")
    print("✅ Random-start 12-GW window test passed.")


Run 0: start=18, end=29, gw_steps≈12 OK
Run 1: start=1, end=12, gw_steps≈12 OK
Run 2: start=16, end=27, gw_steps≈12 OK
Run 3: start=28, end=38, gw_steps≈11 OK
Run 4: start=25, end=36, gw_steps≈12 OK
✅ Random-start 12-GW window test passed.
